In [ ]:

import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.5.5'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [3,962 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,535 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Pac

In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read the home_sales_revised.csv from the provided AWS S3 bucket location into a PySpark DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"


In [ ]:
# 2. Create a temporary view of the DataFrame.
spark.sparkContext.addFile(url)
sale_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True, ignoreLeadingWhiteSpace=True)
sale_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [ ]:
# 2. Create a temporary view of the DataFrame.
sale_df.createOrReplaceTempView('home_sales')

In [ ]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?

a = """
SELECT
YEAR(date) AS YEAR,
ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
WHERE bedrooms = 4
GROUP BY YEAR
ORDER BY YEAR DESC
"""
spark.sql(a).show()

+----+-------------+
|YEAR|AVERAGE_PRICE|
+----+-------------+
|2022|    296363.88|
|2021|    301819.44|
|2020|    298353.78|
|2019|     300263.7|
+----+-------------+



In [ ]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
b = """
SELECT
YEAR(date_built) AS YEAR,
ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
WHERE bedrooms = 3 and bathrooms=3
GROUP BY YEAR(date_built)
ORDER BY YEAR DESC
"""
spark.sql(b).show()

+----+-------------+
|YEAR|AVERAGE_PRICE|
+----+-------------+
|2017|    292676.79|
|2016|    290555.07|
|2015|     288770.3|
|2014|    290852.27|
|2013|    295962.27|
|2012|    293683.19|
|2011|    291117.47|
|2010|    292859.62|
+----+-------------+



In [ ]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
c = """
SELECT
YEAR(date_built) AS YEAR_BUILT,
ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
WHERE bedrooms = 3
and bathrooms = 3
and sqft_living >= 2000
and floors = 2
GROUP BY YEAR_BUILT
ORDER BY YEAR_BUILT DESC
"""
spark.sql(c).show()

+----------+-------------+
|YEAR_BUILT|AVERAGE_PRICE|
+----------+-------------+
|      2017|    280317.58|
|      2016|     293965.1|
|      2015|    297609.97|
|      2014|    298264.72|
|      2013|    303676.79|
|      2012|    307539.97|
|      2011|    276553.81|
|      2010|    285010.22|
+----------+-------------+



In [ ]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

d = """
SELECT
view,
ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view desc
"""
spark.sql(d).show()



print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

--- 1.1311416625976562 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.
spark.sql('cache table home_sales')

DataFrame[]

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [ ]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

e = """
SELECT
view,
ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
GROUP BY view
HAVING AVG(price) >= 350000
ORDER BY view desc
"""
spark.sql(e).show()



print("--- %s seconds ---" % (time.time() - start_time))


+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

--- 0.7106127738952637 seconds ---


In [ ]:
sale_df.show() and sale_df.printSchema()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [ ]:
sale_df.filter(sale_df.date_built.isNull()).show()


+---+----+----------+-----+--------+---------+-----------+--------+------+----------+----+
| id|date|date_built|price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+---+----+----------+-----+--------+---------+-----------+--------+------+----------+----+
+---+----+----------+-----+--------+---------+-----------+--------+------+----------+----+



In [ ]:
sale_df.select("date_built").distinct().show()


+----------+
|date_built|
+----------+
|      2016|
|      2012|
|      2017|
|      2014|
|      2013|
|      2011|
|      2015|
|      2010|
+----------+



In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data

sale_df.write.parquet('parquet_sales', mode='overwrite')
sale_df.write.partitionBy("date_built").mode("overwrite").parquet("parquet_sales_parts")

In [ ]:
# 11. Read the formatted parquet data.
#p_homes_df = spark.read.parquet('p_home_sales')
p_df_p = spark.read.parquet('parquet_sales_parts')


In [ ]:
# 12. Create a temporary table for the parquet data.
p_df_p.createOrReplaceTempView('p_sales_p')


In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

spark.sql("""

    SELECT ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    WHERE price >= 350000
    GROUP BY view

""").show(truncate=False)


print("--- %s seconds ---" % (time.time() - start_time))

+----------+
|avg_price |
+----------+
|403005.77 |
|788128.21 |
|404673.3  |
|798684.82 |
|399548.12 |
|397771.65 |
|750537.94 |
|396964.5  |
|1072285.2 |
|752861.18 |
|767036.67 |
|398867.6  |
|397862.0  |
|401419.75 |
|791453.0  |
|398592.71 |
|402124.62 |
|402022.68 |
|1056336.74|
|399586.53 |
+----------+
only showing top 20 rows

--- 0.7597565650939941 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql('uncache table home_sales')


DataFrame[]

In [ ]:
# 15. Check if the home_sales is no longer cached
if spark.catalog.isCached('home_sales'):
  print('home_sales is cached')
else:
  print('all clear')

all clear
